In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

In [2]:
import pandas as pd
pd.set_option('display.max_row',300)

# 几种不同的agent类型
- ZERO_SHOT_REACT_DESCRIPTION
- CHAT_ZERO_SHOT_REACT_DESCRIPTION
- CONVERSATIONAL_REACT_DESCRIPTION
- CHAT_CONVERSATIONAL_REACT_DESCRIPTION
- OPENAI_FUNCTIONS
- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
<hr>

ZERO_SHOT_REACT_DESCRIPTION
零样本增强式生成，即在没有示例的情况下可以自主的进行对话的类型。https://blog.csdn.net/zcyzcyjava/article/details/127006287 [零样本、单样本、少样本]

In [3]:
from langchain.llms import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"

llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)
#agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")


verbose=True tags=['zero-shot-react-description'] agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'), llm=OpenAI(client=<openai.resources.comp

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


CHAT_ZERO_SHOT_REACT_DESCRIPTION 使用了chatmodel

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
#agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


verbose=True tags=['chat-zero-shot-react-description'] agent=ChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Search, Calculator\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_N

CONVERSATIONAL_REACT_DESCRIPTION 一个对话型的agent，这个agent要求与memory一起使用

In [5]:
from langchain.llms import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)

memory=ConversationBufferMemory(memory_key='chat_history') verbose=True tags=['conversational-react-description'] agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Ass

In [6]:
agent.run("hi i am tomie")



> Entering new AgentExecutor chain...


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



AI: Hello Tomie, it's nice to meet you! How can I assist you today?

> Finished chain.


"Hello Tomie, it's nice to meet you! How can I assist you today?"

In [7]:
agent.run("what is my name?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "what is my name"
Observation: ['My Name Is type: Song by Eminem.', 'My Name Is kgmid: /g/1s05n4sxn.', 'My Name Is album: The Slim Shady LP.', 'My Name Is released: 1999.', 'My Name Is artist: Eminem.', 'My Name Is awards: Grammy Award for Best Rap Solo Performance, MTV Video Music Award for Best New Artist.', 'My Name Is genres: Spoken Word, Hip-Hop/Rap.', 'Disney+ is the only place to stream your favorites from Disney, Pixar, Marvel, Star Wars, National Geographic and more.', '"What\'s my name? What\'s my name? Say it louder! Sing along to ""What\'s My Name"" with China Anne McClain from Descendants 2 and watch ...', "REMASTERED IN HD! Get Rihanna's eighth studio album ANTI now: Download on TIDAL: http://smarturl.it/downloadANTI Stream on TIDAL: ...", 'Disney+ is the only place to stream your favorites from Disney, Pixar, Marvel, Star Wars, National Geographic and more.', '"What

'Your name is Tomie.'

In [8]:
agent.run("有什么好吃的泰国菜可以推荐给我吗?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Thai food recommendations

ValueError: Got error from SerpAPI: Your account has run out of searches.

In [ ]:
agent.run("这些我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "What is the last letter of my name?" and "Who won the 1998 World Cup?"
Observation: ['The final was held on 12 July 1998 at the Stade de France, Saint-Denis. France defeated holders Brazil 3–0, with two goals from Zinedine Zidane and a stoppage ...', 'Here is every World Cup winner, runner up and third place team dating back to the first tournament in 1930.', "Zidane was named the man of the match, while Ronaldo was awarded the Golden Ball as FIFA's outstanding player of the tournament. Following the win, hundreds of ...", 'Take a look through history at each of the first 20 World Cup champions.', "Over the course of history, only one host had reached the final and not won the match – that was Brazil in 1950. Since the World Cup's inception ...", 'Staging the World Cup for the second time, France reached their first Final, against four-time champions and hot favourites Brazil no 

'The last letter of your name is "e". And the winner of the 1998 World Cup was France.'

In [ ]:
agent.run("中国陕西西安现在的气温多少？截止目前我们聊了什么？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: What is the current temperature in Xi'an, Shaanxi, China?
Observation: {'type': 'weather_result', 'temperature': '31', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '94%', 'wind': '2 mph', 'location': "Xi'An, Shaanxi, China", 'date': 'Wednesday 10:00 PM', 'weather': 'Mostly cloudy'}
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: What have we talked about so far?
Observation: ["As asked by you, the correct sentence is “we did talk”. The concept for this is we don't use second form of verb with “did”. We mostly use it ...", "... I run out of oxygen When I run away to Mars I can't tell which way's home I've been gone for so long It's an empty world Up here I skip ...", 'High quality example sentences with “we have discussed so far” in context from reliable sources - Ludwig is the linguistic search engine that helps you to ...', "So far, we've talked abo

"We have talked about a variety of topics, including your name, Thai food recommendations, the current temperature in Xi'an, and the winner of the 1998 World Cup. Is there anything else you would like to discuss?"

CHAT_CONVERSATIONAL_REACT_DESCRIPTION 使用了chatmodel

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)
print(agent.agent.llm_chain.prompt.messages[3])

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['chat-conversational-react-description'] agent=ConversationalChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Ass

In [ ]:
agent.run(input="hi i am tony")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello Tony, how can I assist you today?"
}
```

> Finished chain.


'Hello Tony, how can I assist you today?'

In [ ]:
agent.run(input="what is my name?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Your name is Tony."
}
```

> Finished chain.


'Your name is Tony.'

In [ ]:
agent.run(input="有什么好吃的泰国菜给我推荐一下？")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "推荐的泰国菜"
}
```
Observation: ['要说到泰菜里面Vivian最喜欢的一定要属菠萝炒饭啦！泰国香米的香气，搭配菠萝独有的酸酸甜甜的味道，真的是升华了整道炒饭的口感！而且这道菜营养均衡，有 ...', '在泰国必吃的11道美味泰国菜 · 1.冬阴功汤 · 2.青木瓜沙拉 · 3.泰国娘惹咖喱酱 · 4.泰式炒河粉 · 5.Pad Kra Pow Gai · 6.菠萝泰式炒饭 · 7.绿咖喱 · 8.辣牛肉沙律.', '青木瓜沙拉(ส้มตำ) · 泰式柚子沙拉(ยำส้มโอ) · 泰式春卷(ปอเปี๊ยะสด) · 清汤冬阴功(ต้มยำกุ้งน้ำใส) · 海南鸡饭(ข้าวมันไก่) · 柠檬蒸鲈鱼(ปลากะพงนึ่งมะนาว) · 泰式椰汁煮 ...', '大家喜爱的泰国菜有冬阴功汤，Pad Thai， 青木瓜色拉，咖喱等。洛杉矶这里的泰国菜种类主要有两种；一种是地道的泰式风味，另一种则是亚洲融合菜。比如 ...', '同样是泰国代表菜的木瓜沙拉，发源于泰国东北，老挝等地。 决定一份木瓜沙拉的好坏主要是木瓜的新鲜程度，甜度，辣椒的辣度和盐的比例。 味道非常爽口，是解腻良菜。 重点推荐。', '10道必尝的泰国菜 · Larb Moo · Phad Thai · Khao Soi · Kra Pao Moo Krob · Som Tam · Gai Pad Med Ma Muang · Hor Mok Pla ...', "9家纽约必吃泰国菜 · Thai Villa · Fish Cheeks · LumLum · Up Thai · Wayla · Soothr's · The Tyger · Thai Diner.", '去泰国餐厅10大必点菜! · 一、青木瓜色拉. 很多人都超级爱青木瓜色拉的酸辣爽脆，凉拌青木瓜色拉，不仅保留了青木瓜的原始营养状态，加入了青瓜丝的清新， ...', '曼谷當地必吃美食 · 辣蝦湯 · 辣味青木瓜沙拉 · 椰子燉雞湯 · 紅咖哩 · 泰式炒河粉 · 泰式炒飯 · 打拋豬肉 · 綠咖哩雞 ...', 

'泰国菜中有许多美味的选择。比如菠萝炒饭，它将泰国香米的香气与菠萝的酸甜味道完美结合，营养均衡。其他推荐的泰国菜包括冬阴功汤、青木瓜沙拉、泰式炒河粉、Pad Kra Pow Gai（罗勒炒鸡）、绿咖喱、辣牛肉沙律等。这些菜肴不仅味道丰富，而且色香味俱佳，值得尝试。'

In [ ]:
agent.run(input="我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "1998年世界杯冠军"
}
```
Observation: France national football team
Thought:```json
{
    "action": "Final Answer",
    "action_input": "你名字的最后一个字母是 Y。1998年的世界杯冠军是法国国家足球队。"
}
```

> Finished chain.


'你名字的最后一个字母是 Y。1998年的世界杯冠军是法国国家足球队。'

OPENAI_FUNCTIONS，使用openai的函数调用来实现的，只支持openai模型。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.prompt.messages)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['openai-functions'] agent=OpenAIFunctionsAgent(llm=ChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4-1106-preview', temperature=0.0, openai_api_key='sk-9npZpELUzM0IxRU1B79bF5B9FfC04bDeA730F4090dF782A3', openai_api_base='https://ai-yyds.com/v1', openai_proxy='https://ai-yyds.com/v1'), tools=[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
agent.run(input="今天比特币的价格是多少？它的价格开平方是多少？")

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `Search` with `
Bitcoin price today`


42,449.40
Invoking: `Calculator` with `sqrt(42449.40)`


Answer: 206.03252170470563今天比特币的价格是 $42,449.40。它的价格开平方后大约是 $206.03。

> Finished chain.


'今天比特币的价格是 $42,449.40。它的价格开平方后大约是 $206.03。'

STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 对输出做了结构化处理

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[1].prompt.template)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['structured-chat-zero-shot-react-description'] agent=StructuredChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {{\'tool_input\': {{\'type\': \'string\'}}}}\nCalculator: Useful for when you need to answer questions about math., args: {{\'tool_input\': {{\'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Search, Calculator\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
agent.invoke({"input":"what is langchain?"})



> Entering new AgentExecutor chain...
Thought: Langchain is not a widely recognized term that I am familiar with, and it may refer to a specific product, service, or concept that has emerged after my last update. I will use the search tool to find the most current information on "langchain."

Action:
```
{
  "action": "Search",
  "action_input": {"type": "langchain"}
}
```
Observation: ["LangChain is a framework designed to simplify the creation of applications using large language models. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.", 'LangChain initial_release_date: October 2022.', 'LangChain developer_s: Harrison Chase.', 'LangChain license: MIT License.', 'LangChain stable_release: 0.0.351 / 18 December 2023; 26 days ago.', 'LangChain written_in: Python and JavaScript.', 'https://www.langchain.com/', 'https://github.com/langchai

{'input': 'what is langchain?',
 'chat_history': [HumanMessage(content='what is langchain?'),
  AIMessage(content='LangChain is a framework designed to simplify the creation of applications using large language models. It is used for tasks like document analysis, summarization, chatbots, and code analysis. LangChain was initially released in October 2022 by developer Harrison Chase and is licensed under the MIT License. The latest stable release is version 0.0.351, which came out on December 18, 2023. It is written in Python and JavaScript. For more information, you can visit the official website at www.langchain.com or its GitHub repository at github.com/langchain-ai/langchain.')],
 'output': 'LangChain is a framework designed to simplify the creation of applications using large language models. It is used for tasks like document analysis, summarization, chatbots, and code analysis. LangChain was initially released in October 2022 by developer Harrison Chase and is licensed under the 